In [ ]:
pip install tensorflow opencv-python matplotlib

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Load the dataset
data_dir = "/path/to/leapgestrecog"
gestures = os.listdir(data_dir)
image_data = []
labels = []

for gesture in gestures:
    gesture_path = os.path.join(data_dir, gesture)
    for filename in os.listdir(gesture_path):
        img_path = os.path.join(gesture_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))
        image_data.append(img)
        labels.append(gesture)

# Convert the lists to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Normalize pixel values to be between 0 and 1
image_data = image_data / 255.0

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = to_categorical(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

# Build the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(gestures), activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train.reshape(-1, 64, 64, 1), y_train, epochs=10, validation_data=(X_test.reshape(-1, 64, 64, 1), y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test.reshape(-1, 64, 64, 1), y_test)
print(f'Test Accuracy: {test_acc}')

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()
